insert here acedemic token and query info

In [ ]:
Token = "***************************************************************************" #insert here acedemic token

In [ ]:
[lat, lon, radius, start_time, end_time]=['12.2558141', '41.9097306', '10', '2020-01-01T00:00:00.001', '2021-01-01T00:00:00.001']

import modules

In [ ]:
import requests
import pandas as pd
import time

define functions

In [ ]:
def getTwitterPost(lat, lon, radius, start_time, end_time):
    query='point_radius:[' + lon + ' ' + lat + ' ' + radius + 'km]'
    baseUrl = "https://api.twitter.com/2/tweets/search/all?max_results=500&tweet.fields=geo&query=has:geo "+query+"&start_time="+start_time+"&end_time="+end_time
    headers = {"Authorization": "Bearer "+Token}
    tweets=[]
    
    # we do first call to get first next_token
    resp = requests.get(
        baseUrl, headers=headers).json()
    data = []
    if('data' in resp):
        data = resp['data']
        tweets = _mergeTweets(tweets, data)
    # we iterate until no next_token (end of pagination)
    if 'meta' in resp and 'next_token' in resp['meta']:
        next_token = resp['meta']['next_token']
        while True:
            time.sleep(3) #api rate limit 300 per 15 min
            resp = requests.get(baseUrl+"&next_token="+next_token, headers=headers).json()
            if('data' in resp):
                data = resp['data']
                tweets = _mergeTweets(tweets, data)
                if('meta' in resp and 'next_token' in resp['meta']):
                    next_token = resp['meta']['next_token']
                else:
                    break
    df = pd.Dataframe(tweets)
    # split data in two dataframe: one with lat e long and one without
    dfNOLatLon = df[df['longitude']=='']
    dfLatLon = df[df['longitude']!='']
    # create a distinct of place_ids to find use 1.1 api to get lat and long
    placesToFind=list(set(dfNOLatLon['place_id'].to_list()))
    dfPlace = findLatLon(places, headers)
    # drop empty columns and join with dataframe without lat and log with dataframe with informations
    dfNOLatLon=dfNOLatLon.drop(columns=['longitude', 'latitude'], inplace=True)
    dfNOLatLon=dfNOLatLon.merge(dfPlace, on='place_id', how='left')
    # concat and export to csv file
    dfFinal=pd.concat([dfLatLon, dfNOLatLon])
    dfFinal.to_csv('tweets.csv')

# extract data from response and concat with stored tweets
def _mergeTweets(tweets, data):
    tweetsNoCorD = []
    for el in data:
        tweet = {}
        #sometimes has:geo return post with only place_id we assigned empty string to filter in second step
        if('geo' in el and 'coordinates' in el['geo'] and el['geo']['coordinates']['type'] == 'Point' and 'place_id' in el['geo']):
            tweet['longitude'] = str(
                el['geo']['coordinates']['coordinates'][0])
            tweet['latitude'] = str(el['geo']['coordinates']['coordinates'][1])
            tweet['place_id'] = el['geo']['place_id']
        elif('geo' in el and 'place_id' in el['geo']):
            tweet['place_id'] = el['geo']['place_id']
            tweet['longitude'] = ''
            tweet['latitude'] = ''
            tweets.append(tweet)
    return tweets

### create dataframe with place_id, latitude and longitude using twitter 1.1 api from list of place_id
def findLatLon(places, headers):
    placeList=[]
    baseUrl = "https://api.twitter.com/1.1/geo/id/"
    for i in range(0, len(places), 70):
        placesInfo = [requests.get(baseUrl+id+".json", headers=headers).json() for id in places[i:i+70]]
        for place in placesInfo:
            if('geometry' in place and 'type' in place['geometry'] and place['geometry']['type'] == "Point"):
                placeList.append({'place_id': place['id'], 'longitude': place['geometry']['coordinates'][0], 'latitude': place['geometry']['coordinates'][1]})
            else:
                #sometimes it return a polygon so we use centroid coordinates
                placeList.append(
                    {'place_id': place['id'], 'longitude': place['centroid'][0], 'latitude': place['centroid'][1]})
        time.sleep(900) #api rate limit 70 per 15 mins
    return pd.DataFrame(placeList)
                

In [ ]:
getTwitterPost(lat, lon, radius, start_time, end_time)